# Навчання моделі (48 годин контексту → 24 години прогнозу)

Цей ноутбук:
1) готує `X` та `y` (вікна 48→24)  
2) навчає модель Keras  
3) зберігає модель у `artifacts/keras_model.keras`  
4) зберігає `artifacts/scaler.json` (для нормалізації в браузері)


In [ ]:
# !pip install -r ../requirements.txt

from pathlib import Path
import json
import numpy as np
import tensorflow as tf

import sys
sys.path.append(str(Path("..").resolve() / "src"))

from dataset import load_weather_csv, clean_and_fill, make_windows, export_scaler_json, FEATURES

DATA = Path("..") / "data" / "weather_kriviyrih.csv"
OUT = Path("..") / "artifacts"
OUT.mkdir(exist_ok=True)

INPUT_HOURS = 48
HORIZON = 24

tf.__version__


## 1) Підготовка датасету

In [ ]:
df = clean_and_fill(load_weather_csv(DATA))
ds = make_windows(df, input_hours=INPUT_HOURS, horizon_hours=HORIZON)

X, y = ds.X, ds.y
X.shape, y.shape


## 2) Train/Val split

In [ ]:
n = len(X)
n_train = int(n * 0.8)

X_train, y_train = X[:n_train], y[:n_train]
X_val, y_val = X[n_train:], y[n_train:]

X_train.shape, X_val.shape


## 3) Модель (Conv1D → GAP → Dense → 24)

In [ ]:
def build_model(input_hours: int, n_features: int, horizon: int) -> tf.keras.Model:
    inp = tf.keras.Input(shape=(input_hours, n_features), name="x")
    x = tf.keras.layers.Conv1D(64, 5, padding="same", activation="relu")(inp)
    x = tf.keras.layers.Conv1D(64, 5, padding="same", activation="relu")(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    out = tf.keras.layers.Dense(horizon, name="y")(x)
    m = tf.keras.Model(inp, out)
    m.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse", metrics=[tf.keras.metrics.MAE])
    return m

model = build_model(INPUT_HOURS, X.shape[-1], HORIZON)
model.summary()


## 4) Навчання

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(str(OUT / "keras_model.keras"), save_best_only=True),
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=80,
    batch_size=64,
    callbacks=callbacks,
    verbose=1,
)


## 5) Збереження scaler.json

In [ ]:
scaler_pack = export_scaler_json(ds.scaler, input_hours=INPUT_HOURS, horizon_hours=HORIZON)
(OUT / "scaler.json").write_text(json.dumps(scaler_pack, ensure_ascii=False, indent=2), encoding="utf-8")

print("Saved:", OUT / "keras_model.keras")
print("Saved:", OUT / "scaler.json")


## 6) Швидка перевірка: прогноз на одному прикладі

In [ ]:
i = -1
pred = model.predict(X_val[i:i+1], verbose=0)[0]  # 24 значення
true = y_val[i]

pred[:5], true[:5]


## Готово
Далі: `03_model_conversion.ipynb`